# Setup

## Install Packages

In [1]:
# load those needed packages and libraries
import torch
#import optuna
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import rpy2.robjects as ro

from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
#from sklearn.model_selection import train_test_split
#from nltk.translate.bleu_score import SmoothingFunction
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rpy2.robjects import pandas2ri

c:\Users\Bryan Chan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set Device 

In [ ]:
# set the device to cuda if available else cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Datasets

In [4]:
# define Data file directory
data_dir = r'C:\Users\Bryan Chan\Documents\Projects\MA_Thesis\data'

# load the data 'df_LibDem.csv'
df_LibDem = pd.read_csv(data_dir + r'\df_LibDem.csv')

In [6]:
# What is the average length of contents in the 'text' column
df_LibDem['text'].str.split().str.len().mean()



224.1978030886466

# Fine-Tune Models

Models: T5 or BERT


## Training with T5

In [ ]:
# Load pre-trained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Load dataset (this should include CoT examples)
dataset = load_dataset('path_to_your_dataset')

# Preprocess the dataset
def preprocess_data(examples):
    inputs = examples['input']
    cot_steps = examples['cot']  # Chain of Thought steps
    outputs = examples['output']

    model_inputs = tokenizer(inputs + cot_steps, max_length=512, 
                             truncation=True, 
                             padding='max_length', return_tensors="pt")
    labels = tokenizer(outputs, max_length=128, 
                       truncation=True, 
                       padding='max_length', return_tensors="pt")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = dataset['train'].map(preprocess_data, batched=True)
val_dataset = dataset['validation'].map(preprocess_data, batched=True)

# Define training arguments
training_args = TrainingArguments(
    data_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

## Training with BERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare your data
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

# Assuming 'speeches' is a pandas DataFrame with your text data
speeches['text'] = speeches['filename'].apply(lambda x: open(x).read())
dataset = speeches['text']
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset))

# Define the training arguments
training_args = TrainingArguments(
    data_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create the Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

# Counterfactual Sampling

In [ ]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self._number_of_visits = 0.
        self._results = 0.

    def expand(self):
        # Implement your expansion logic here
        pass

    def rollout(self):
        # Implement your rollout logic here
        pass

    def backpropagate(self, result):
        self._number_of_visits += 1.
        self._results += result
        if self.parent:
            self.parent.backpropagate(result)

    def is_fully_expanded(self):
        return len(self.children) > 0

    def best_child(self):
        # Implement your selection logic here
        pass

def MCTS(root, iterations):
    for _ in range(iterations):
        node = root
        while node.is_fully_expanded():
            node = node.best_child()
        if not node.is_terminal():
            node.expand()
        result = node.rollout()
        node.backpropagate(result)
    return root.best_child()
